# Mistral Modelleri ile İnşa Etme

## Giriş

Bu ders şunları kapsayacaktır:  
- Farklı Mistral Modellerini keşfetmek  
- Her modelin kullanım durumlarını ve senaryolarını anlamak  
- Kod örnekleri ile her modelin benzersiz özelliklerini göstermek.


## Mistral Modelleri

Bu derste, 3 farklı Mistral modelini keşfedeceğiz:  
**Mistral Large**, **Mistral Small** ve **Mistral Nemo**.

Bu modellerin her biri Github Model pazarında ücretsiz olarak mevcuttur. Bu not defterindeki kod, bu modelleri kullanarak çalıştırılacaktır. Github Modellerini kullanarak [AI modelleri ile prototip oluşturma](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) hakkında daha fazla bilgi burada.


## Mistral Large 2 (2407)
Mistral Large 2 şu anda Mistral'ın amiral gemisi modeli olup kurumsal kullanım için tasarlanmıştır.

Model, orijinal Mistral Large'a göre bir yükseltme sunar:
- Daha Büyük Bağlam Penceresi - 128k vs 32k
- Matematik ve Kodlama Görevlerinde Daha İyi Performans - %76,9 ortalama doğruluk vs %60,4
- Artan çok dilli performans - diller arasında: İngilizce, Fransızca, Almanca, İspanyolca, İtalyanca, Portekizce, Hollandaca, Rusça, Çince, Japonca, Korece, Arapça ve Hintçe bulunur.

Bu özelliklerle Mistral Large şu alanlarda üstünlük sağlar:
- *Retrieval Augmented Generation (RAG)* - daha büyük bağlam penceresi sayesinde
- *Fonksiyon Çağrısı* - bu model, dış araçlar ve API'lerle entegrasyona olanak tanıyan yerel fonksiyon çağrısına sahiptir. Bu çağrılar paralel olarak veya ardışık sırayla yapılabilir.
- *Kod Üretimi* - bu model Python, Java, TypeScript ve C++ üretiminde üstün performans gösterir.


### Mistral Large 2 kullanarak RAG Örneği


Bu örnekte, bir metin belgesi üzerinde RAG deseni çalıştırmak için Mistral Large 2 kullanıyoruz. Soru Korece yazılmıştır ve yazarın üniversite öncesi faaliyetleri hakkında bilgi istemektedir.

Metin belgesinin ve sorunun gömme vektörlerini oluşturmak için Cohere Embeddings Modeli kullanılır. Bu örnek için, vektör deposu olarak faiss Python paketi kullanılır.

Mistral modeline gönderilen istem, hem soruları hem de soruya benzer şekilde alınan parçaları içerir. Model daha sonra doğal dilde bir yanıt sağlar.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small  
Mistral Small, premier/enterprise kategorisi altında Mistral model ailesinin başka bir modelidir. Adından da anlaşılacağı gibi, bu model Küçük Dil Modeli (SLM)dir. Mistral Small kullanmanın avantajları şunlardır:  
- Mistral Large ve NeMo gibi Mistral LLM'lerine kıyasla maliyet tasarrufu - %80 fiyat düşüşü  
- Düşük gecikme - Mistral'ın LLM'lerine kıyasla daha hızlı yanıt  
- Esnek - gerekli kaynaklar konusunda daha az kısıtlama ile farklı ortamlarda dağıtılabilir.  

Mistral Small şunlar için idealdir:  
- Özetleme, duygu analizi ve çeviri gibi metin tabanlı görevler.  
- Maliyet etkinliği nedeniyle sık istek yapılan uygulamalar  
- İnceleme ve kod önerileri gibi düşük gecikmeli kod görevleri  


## Mistral Small ve Mistral Large Karşılaştırması

Mistral Small ve Large arasındaki gecikme farklarını göstermek için aşağıdaki hücreleri çalıştırın.

Aynı istem üzerinde yanıt süreleri arasında 3-5 saniye fark görmelisiniz. Ayrıca yanıt uzunlukları ve stiline de dikkat edin.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Bu derste tartışılan diğer iki modele kıyasla, Mistral NeMo Apache2 Lisansına sahip tek ücretsiz modeldir.

Mistral'ın önceki açık kaynak LLM'si olan Mistral 7B'nin bir yükseltmesi olarak görülmektedir.

NeMo modelinin bazı diğer özellikleri şunlardır:

- *Daha verimli tokenizasyon:* Bu model, daha yaygın kullanılan tiktoken yerine Tekken tokenizer kullanmaktadır. Bu, daha fazla dil ve kod üzerinde daha iyi performans sağlar.

- *İnce ayar:* Temel model ince ayar için kullanılabilir. Bu, ince ayarın gerekebileceği kullanım durumları için daha fazla esneklik sağlar.

- *Yerel Fonksiyon Çağrısı* - Mistral Large gibi, bu model fonksiyon çağrısı üzerinde eğitilmiştir. Bu, onu bunu yapan ilk açık kaynak modellerden biri olarak benzersiz kılar.


## Mistral NeMo

Bu derste tartışılan diğer iki modele kıyasla, Mistral NeMo Apache2 Lisansına sahip tek ücretsiz modeldir.

Mistral'ın önceki açık kaynak LLM'si olan Mistral 7B'nin bir yükseltmesi olarak görülmektedir.

NeMo modelinin bazı diğer özellikleri şunlardır:

- *Daha verimli tokenizasyon:* Bu model, daha yaygın kullanılan tiktoken yerine Tekken tokenizer'ı kullanır. Bu, daha fazla dil ve kod üzerinde daha iyi performans sağlar.

- *İnce ayar:* Temel model ince ayar için kullanılabilir. Bu, ince ayarın gerekebileceği kullanım durumları için daha fazla esneklik sağlar.

- *Yerel Fonksiyon Çağrısı* - Mistral Large gibi, bu model fonksiyon çağrısı üzerinde eğitilmiştir. Bu, onu bunu yapan ilk açık kaynak modellerden biri olarak benzersiz kılar.


### Tokenlaştırıcıları Karşılaştırma

Bu örnekte, Mistral NeMo'nun tokenlaştırmayı Mistral Large ile nasıl ele aldığını inceleyeceğiz.

Her iki örnek de aynı istemi alır ancak NeMo'nun Mistral Large'a kıyasla daha az token döndürdüğünü görmelisiniz.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Öğrenme burada bitmiyor, yolculuğa devam edin

Bu dersi tamamladıktan sonra, Generative AI bilginizi geliştirmeye devam etmek için [Generative AI Öğrenme koleksiyonumuza](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) göz atın!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, AI çeviri servisi [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba gösterilse de, otomatik çevirilerin hatalar veya yanlışlıklar içerebileceğini lütfen unutmayınız. Orijinal belge, kendi dilinde yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımı sonucu oluşabilecek yanlış anlamalar veya yorum hatalarından sorumlu değiliz.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
